In [96]:
import psycopg2
import pandas.io.sql as psql
import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
import re
import pandas as pd
from datetime import datetime
import datetime as dt

current_year = 0
current_week = 0


file = 'kvat_sales.xlsx'

old = pd.read_excel(f'{file}', skiprows=1)
date = old.iloc[0, 4]

A = datetime.strptime(date, '%m/%d/%Y')
store_year = A.year


# this is finding the current week for the stores
# day = A.day
# month = A.month
# year = A.year

# current_week = date(A).isocalendar()[1]

# recreating data frame so I don't have to redefine the column names
old = pd.read_excel(f'{file}', skiprows=3)


old['qty'] = old['Units Sold'] + old['Units Sold.1'] + old['Units Sold.2'] + old['Units Sold.3'] + old[
    'Units Sold.4'] + old['Units Sold.5'] + old['Units Sold.6']

old['sales'] = old['Sales $'] + old['Sales $.1'] + old['Sales $.2'] + old['Sales $.3'] + old['Sales $.4'] + old[
    'Sales $.5'] + old['Sales $.6']

# drops last row due to grand total. after dropping it,
# store numbers can be then converted to int values

old.drop(old.tail(1).index, inplace=True)  # drop last n rows

old['store_number'] = old['Store Number'].astype(float).astype(np.int64)

# upc values are converted from float to int64 (db requires this data type)

old['UPC'] = old['UPC'].astype(float).astype(np.int64)

# seting transition date range and store_week values for data and store_type
old['transition_year'] = transition_year
old['transition_season'] = f'{transition_season}'

old['store_week'] = date
salesdata['store_week']= pd.to_datetime(salesdata['store_week'])
date = old.loc[0, 'store_week']
# current_week = date.week-1
# current_year = date.year


old['store_type'] = 'kvat'
old['store_year'] = store_year
old['current_year'] = current_year
old['current_week'] = current_week

old = old.rename(columns={'UPC': 'upc'})

salesdata = old[['transition_year',
                 'transition_season',
                 'store_year',
                 'store_week',
                 'store_number',
                 'upc',
                 'sales',
                 'qty',
                 'current_year',
                 'current_week',
                 'store_type'

                 ]]

salesdata


C:\Users\User1\AppData\Local\Temp/ipykernel_22256/131036247.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salesdata['store_week']= pd.to_datetime(salesdata['store_week'])


,transition_year,transition_season,store_year,store_week,store_number,upc,sales,qty,current_year,current_week,store_type
0,2022,SS,2022,6/27/2022,211,81031202441,14.99,1,0,0,kvat
1,2022,SS,2022,6/27/2022,211,81031202427,53.97,3,0,0,kvat
2,2022,SS,2022,6/27/2022,214,81031202369,27.96,4,0,0,kvat
3,2022,SS,2022,6/27/2022,214,81031202370,13.98,2,0,0,kvat
4,2022,SS,2022,6/27/2022,214,81031202449,16.99,1,0,0,kvat
...,...,...,...,...,...,...,...,...,...,...,...
299,2022,SS,2022,6/27/2022,2446,81031202418,29.90,2,0,0,kvat
300,2022,SS,2022,6/27/2022,2446,81031202427,53.85,3,0,0,kvat
301,2022,SS,2022,6/27/2022,2457,81031202449,33.90,2,0,0,kvat
302,2022,SS,2022,6/27/2022,2457,81031202419,16.95,1,0,0,kvat


In [97]:
# salesdata['store_week']= pd.to_datetime(salesdata['store_week'])
test = salesdata.loc[0,'store_week']
type(test)
# salesdata
# store_week = a.week-1
# store_year = a.year
# print(store_week,store_year)
# store_week = a + dt.timedelta(days=1).week
# store_week

str

In [61]:
#transform jewel program

import re
import pandas as pd
import datetime as dt
import psycopg2

import pandas.io.sql as psql

import numpy as np
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)

store_type_input ='jewel'
file = 'Jewel Sales.xlsx'
transition_year = 2022
transition_season = 'SS'
connection = psycopg2.connect(database=f"{store_type_input}", user="postgres", password="winwin", host="localhost")    


old = pd.read_excel(f'{file}', sheet_name='Product Scan', skiprows=1)

#verify data

verify =old.loc[0,'Division']
if 'JEWEL' in verify:
    verify = 'pass'
    
if verify == 'pass':
    

    # inserting neccessary collumns for sales insert function
    old['transition_year'] = transition_year
    old['transition_season'] = transition_season
    old['store_year'] = 0
    old['current_year'] = 0
    old['current_week'] = 0



    # Grabs only the neccessary collumns and reformats them into the neccesary df format for the sale insertert function
    old = old[['transition_year',
               'transition_season',
               'store_year',
               'Day',
               'Store',
               'UPC',
               'Sum Net Amount',
               'Sum Item Quantity',
               'current_year',
               'current_week',
               'Division']]

    old = old.rename(columns={
        'Division': 'store_type',
        'Store': 'store_number',
        'UPC': 'upc',
        'Sum Net Amount': 'sales',
        'Sum Item Quantity': 'qty',
        'Day': 'store_week'
    })

    # extracting only the store division name from colllumn and then lower caseing it so it can pass the db security check
    old['store_type'] = old.store_type.str.replace('[^a-zA-Z]', '')
    old['store_type'] = old.store_type.str.lower()

    # Find the last date in the db

    date = psql.read_sql('select max(store_week) from sales2', connection)
    date = pd.Timestamp(date.iloc[0, 0])
    


    # sorts dates in order 
    old = old.sort_values(by='store_week', ascending=True)
    
    
    # using that date select the data from that date to present day in the sales sheet.
    filt = (old['store_week'] >= date)
    old = old.loc[filt]
    
    
    #assigns store year, current year, and current week using the store_week column. 
    # Note that when finding the week number for the current week the week begins on Sunday and ends on Saturday
    # this is per the CVS Schedule 
    
    old['store_year']= old['store_week'].dt.year
    old['current_year']= old['store_week'].dt.year
    old['current_week']= old['store_week'].apply(lambda x: (x + dt.timedelta(days=1)).week)



salesdata = old[['transition_year',
                 'transition_season',
                 'store_year',
                 'store_week',
                 'store_number',
                 'upc',
                 'sales',
                 'qty',
                 'current_year',
                 'current_week',
                 'store_type']]
# salesdata.head(50)
salesdata = salesdata.reset_index(drop=True)
salesdata


C:\Users\User1\AppData\Local\Temp/ipykernel_22256/2606181901.py:64: FutureWarning: The default value of regex will change from True to False in a future version.
  old['store_type'] = old.store_type.str.replace('[^a-zA-Z]', '')


,transition_year,transition_season,store_year,store_week,store_number,upc,sales,qty,current_year,current_week,store_type
0,2022,SS,2022,2022-07-04,3278,81031202385,19.99,1,2022,27,jewel
1,2022,SS,2022,2022-07-04,3170,81031202441,39.98,2,2022,27,jewel
2,2022,SS,2022,2022-07-04,3170,81031202420,99.95,5,2022,27,jewel
3,2022,SS,2022,2022-07-04,3170,81031202449,19.99,1,2022,27,jewel
4,2022,SS,2022,2022-07-04,3178,81031202427,39.98,2,2022,27,jewel
...,...,...,...,...,...,...,...,...,...,...,...
178,2022,SS,2022,2022-07-04,3331,81031202372,19.99,1,2022,27,jewel
179,2022,SS,2022,2022-07-04,3343,81031202427,19.99,1,2022,27,jewel
180,2022,SS,2022,2022-07-04,3343,81031202366,7.99,1,2022,27,jewel
181,2022,SS,2022,2022-07-04,4057,81031202367,7.99,1,2022,27,jewel


In [62]:
a=salesdata.loc[0,'store_week']
type(a)

pandas._libs.tslibs.timestamps.Timestamp

In [44]:
#trying to fix week num problem for jewel

from datetime import datetime, timedelta

old = pd.read_excel('jewel_salesdata.xlsx')
old = old.drop(columns=['current_week'])


#creating a collumn where 1 day is subtracted so week num is calculated, weeknum starts from monday not sunday
old['start_date'] = old['store_week'] - timedelta(days=1)

old[['year', 'current_week', 'day']] = old['start_date'].dt.isocalendar()

old['current_week'] = old['current_week'] + 1

old = old.drop(columns =['year','day'])
# old.to_excel('jewel_replacement.xlsx', index = False)

old

# dateTimeInstance = datetime.datetime(2022, 1, 1)
# weeknum =dateTimeInstance.isocalendar()
# weeknum

,transition_date_range,store_year,store_week,store_number,upc,sales,qty,current_year,store_type,start_date,current_week
0,NaN,2021,2021-02-28,52,81031202046,6.99,1,2021,jewel,2021-02-27,9
1,NaN,2021,2021-02-28,185,81031202111,13.00,4,2021,jewel,2021-02-27,9
2,NaN,2021,2021-02-28,3084,81031202111,12.98,2,2021,jewel,2021-02-27,9
3,NaN,2021,2021-03-01,3426,81031202046,6.99,1,2021,jewel,2021-02-28,9
4,NaN,2021,2021-03-01,3426,81031202065,34.95,5,2021,jewel,2021-02-28,9
...,...,...,...,...,...,...,...,...,...,...,...
104063,2022SS,2022,2022-05-22,3191,81031202418,19.99,1,2022,jewel,2022-05-21,21
104064,2022SS,2022,2022-05-22,3191,81031202420,39.98,2,2022,jewel,2022-05-21,21
104065,2022SS,2022,2022-05-22,3741,81031202420,19.99,1,2022,jewel,2022-05-21,21
104066,2022SS,2022,2022-05-22,3770,81031202420,19.99,1,2022,jewel,2022-05-21,21


In [9]:
# code for updating support sheet

from psycopg2 import pool
import pandas as pd
import pandas.io.sql as psql
import datetime
from Import.data_insertion import *


from Sales_Report.Reports.reportsdata import *
year = 2022
week = 25
file='support_sheet'

store_type_input = 'kroger_central'

# store_setting = pd.read_excel(
#     rf'C:\Users\User1\OneDrive - winwinproducts.com\Groccery Store Program\{store_type_input}\{store_type_input}_store_setting.xlsm',
#     sheet_name='Sheet2',
#     header=None,
#     index_col=0,
#     names=('setting', 'values'))

connection = psycopg2.connect(database=f"{store_type_input}",
                                           user="postgres",
                                           password="winwin",
                                           host="localhost")

connection_pool = pool.SimpleConnectionPool(1, 10000, 
                                            database= f"{store_type_input}", 
                                            user="postgres", 
                                            password="winwin", 
                                            host="localhost")


# from store_info import Replenishment

# acme = Replenishment(store_type_input='acme',
#                      current_year = year , 
#                      current_week= week)

# acme.support_import('support_sheet')



itemsupport = pd.read_excel(f'{file}.xlsx')

update= 0
insert = 0

item_support = len(itemsupport)
i = 0

while i < item_support:

    season = itemsupport.loc[i, 'season']
    category = itemsupport.loc[i, 'category']
    type = itemsupport.loc[i, 'type']
    style = itemsupport.loc[i, 'style']
    additional = itemsupport.loc[i, 'additional']
    display_size = itemsupport.loc[i, 'display_size']
    pog_type = itemsupport.loc[i, 'pog_type']
    upc = itemsupport.loc[i, 'upc']
    code = itemsupport.loc[i, 'code']
    code_qb = itemsupport.loc[i, 'code_qb']
    unique_replen_code = itemsupport.loc[i, 'unique_replen_code']
    case_size = itemsupport.loc[i, 'case_size']
    item_group_desc = itemsupport.loc[i, 'item_group_desc']
    item_desc = itemsupport.loc[i, 'item_desc']
    packing = itemsupport.loc[i, 'packing']
    upc_11_digit = str(int(itemsupport.loc[i, 'upc_11_digit']))
    
    
    duplicate_check = psql.read_sql(f"""
                                        select * from item_support2
                                        where code = '{code}' 
                                        """, connection)

    
    if len(duplicate_check) == 1:
    
        
        item_support_update(season,
                        category,
                        type,
                        style,
                        additional,
                        display_size,
                        pog_type,
                        upc,
                        code,
                        code_qb,
                        unique_replen_code,
                        case_size,
                        item_group_desc,
                        item_desc,
                        packing,
                        upc_11_digit,
                        connection_pool)
        update+=1
    
    else:
          
        item_support_insert(season,
                            category,
                            type,
                            style,
                            additional,
                            display_size,
                            pog_type,
                            upc,
                            code,
                            code_qb,
                            unique_replen_code,
                            case_size,
                            item_group_desc,
                            item_desc,
                            packing,
                            upc_11_digit,
                            connection_pool)
        insert+=1
        print(code)
        
        
        
        
    i += 1
print(update,insert)

GMTXT100-AY
GMTXT110-AY
GMRA10210-AY
GMRA10215-AY
GMRA10216-AY
GMFTG100-FW
GMAKH1000-FW
LISDJ203-AY
LISDJ202-AY
LISDJ201-AY
LIODJ303-AY
LIODJ302-AY
LIODJ301-AY
GMXDJ110-AY
342 14


In [8]:
#Inventory Size Build UP

from psycopg2 import pool
import pandas as pd
import pandas.io.sql as psql
import datetime
from Import.data_insertion import *

from Sales_Report.Reports.reportsdata import ReportsData

store_type_input = 'kroger_columbus'

store_setting = pd.read_excel(
    rf'C:\Users\User1\OneDrive - winwinproducts.com\Groccery Store Program\{store_type_input}\{store_type_input}_store_setting.xlsm',
    sheet_name='Sheet2',
    header=None,
    index_col=0,
    names=('setting', 'values'))

connection = psycopg2.connect(database=f"{store_type_input}",
                                           user="postgres",
                                           password="winwin",
                                           host="localhost")

report = ReportsData(store_type_input, store_setting)
on_hand=report.on_hands()
on_hand


# finds all of the diffent items that are located in the store
unique_items= on_hand['item_group_desc'].unique()
A = pd.DataFrame(unique_items, columns=['item_group_desc'])


#Uses the items found at the store and finds the type of item it is (ie legging, Jegging, etc.)
item_type = psql.read_sql(f"""select distinct(item_group_desc), type from item_support2""", connection)

unique_item_type = A.merge(item_type, how='inner', on='item_group_desc')


#filters out only the items that are legging and jeggings
unique_item_type_filt = unique_item_type[(unique_item_type['type']=='Legging') | (unique_item_type['type']=='Jegging')]
unique_item_type_filt = unique_item_type_filt.reset_index(drop=True)

unique_item_type_filt  #LIST WILL BE USED LATER


#on hand table will be using this to add the sizes here
on_hand_size = on_hand #[['store','item_group_desc', 'on_hand']]
on_hand_size[['S-XL','2XL-3XL','S-M','L-XL','S-2XL','S-L','L-2XL']] = 0
on_hand_size= on_hand_size.set_index(['store','item_group_desc'])
on_hand_size

i=0

while i < len(unique_item_type_filt):


    #assigns the item to a variable and then searches the onhand table for that item using that variable
    item_group_desc = unique_item_type_filt.loc[i,'item_group_desc']

    on_hand_filt = on_hand[on_hand['item_group_desc']==f'{item_group_desc}']
    on_hand_store_item_filt = on_hand_filt.reset_index(drop=True)


    #takes that list of stores that have that particular item and then looks at each 
    #store and finds the  on hand for each item upc
    x=0
    while x < len(on_hand_store_item_filt):


        store = on_hand_store_item_filt.loc[x,'store']

        #SQL statement below produces a query that shows the on hands for a store for a particular items by size
        store_item_onhand_by_size =psql.read_sql(f"""

                        WITH

                            /*Finds deliveries for all stores and groups qty by upc and store*/
                            DELIV AS (
                                select STORE, DELIVERY2.UPC, DELIVERY2.CODE, QTY, ITEM_GROUP_DESC, UPC_11_DIGIT  
                                from DELIVERY2
                                INNER JOIN ITEM_SUPPORT2 ON ITEM_SUPPORT2.CODE = DELIVERY2.CODE),

                            /*Finds sales for all stores and groups qty by upc and store*/
                            SALES AS (
                                SELECT DISTINCT(ID), STORE_NUMBER, SALES2.UPC, QTY, ITEM_GROUP_DESC, CODE
                                from SALES2
                                INNER JOIN ITEM_SUPPORT2 ON ITEM_SUPPORT2.UPC_11_DIGIT = SALES2.UPC),

                            DELIV_ITEM AS (
                                SELECT DISTINCT(CODE), UPC_11_DIGIT, SUM(QTY)AS DELIV FROM DELIV
                                WHERE ITEM_GROUP_DESC = '{item_group_desc}' AND STORE = {store}
                                GROUP BY DELIV.CODE,UPC_11_DIGIT),

                            SALES_ITEM AS (
                                SELECT CODE, UPC, SUM(QTY) AS SALES FROM SALES
                                WHERE ITEM_GROUP_DESC = '{item_group_desc}' AND STORE_NUMBER = {store}
                                GROUP BY SALES.CODE, UPC)


                        SELECT size, (DELIV-SALES) AS ON_HAND FROM DELIV_ITEM
                        INNER JOIN SALES_ITEM ON DELIV_ITEM.CODE = SALES_ITEM.CODE
                        Inner join item_size on DELIV_ITEM.CODE = item_size.code

            """,connection)

        #Takes that query generated above and then assigns info to the final table
        y=0
        while y < len(store_item_onhand_by_size):

            size = store_item_onhand_by_size.loc[y,'size']
            size_on_hand = store_item_onhand_by_size.loc[y,'on_hand']
            on_hand_size.loc[(store, item_group_desc),f'{size}']= size_on_hand
            y+=1

        x+=1
    
    i+=1
on_hand_size= on_hand_size.reset_index()

on_hand_size['size sum'] = on_hand_size['S-XL'] + on_hand_size['2XL-3XL'] + on_hand_size['S-M'] + on_hand_size['L-XL'] + on_hand_size['S-2XL'] + on_hand_size['S-L'] + on_hand_size['L-2XL']
on_hand_size[['S-XL %','2XL-3XL %','S-M %','L-XL %','S-2XL %','S-L %','L-2XL %']] = 0

on_hand_size['S-XL %'] = round(on_hand_size['S-XL']/on_hand_size['size sum'],2)
on_hand_size['2xl-3XL %'] = round(on_hand_size['2XL-3XL']/on_hand_size['size sum'],2)
on_hand_size['S-M %'] = round(on_hand_size['S-M']/on_hand_size['size sum'],2)
on_hand_size['L-XL %'] = round(on_hand_size['L-XL']/on_hand_size['size sum'],2)
on_hand_size['S-2XL %'] = round(on_hand_size['S-2XL']/on_hand_size['size sum'],2)
on_hand_size['S-L %'] = round(on_hand_size['S-L']/on_hand_size['size sum'],2)
on_hand_size['L-2XL %'] = round(on_hand_size['L-2XL']/on_hand_size['size sum'],2)



on_hand_size
on_hand_size.to_excel(f'{store_type_input}_size_buildup_report.xlsx', index=False)


In [46]:
on_hand_size.loc[292]

,on_hand,S-XL,2XL-3XL,S-M,L-XL,S-2XL,S-L,L-2XL
item_group_desc,,,,,,,,
PRINT - SSL,40.0,0,0,0,0,0,0,0
HIGH RISE ATHLETIC LEGGINGS,55.0,0,0,0,0,0,0,0
DENIM JEGGING,47.0,0,3,-2,9,0,0,0


In [42]:
for x in store_item_onhand_by_size:
    print(x)

size
on_hand


In [47]:
unique_item_type_filt

,item_group_desc,type
0,SPORT LEGGING,Legging
1,PRINT - SSL,Legging
2,HIGH RISE ATHLETIC LEGGINGS,Legging
3,SOLID - SSL,Legging
4,DENIM JEGGING,Jegging
5,PRINTED DENIM JEGGING,Jegging
